In [1]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython', 'pandas', 'psycopg2-binary']
pipmain(['install'] + packages)

In [2]:
%%capture
from twython import Twython

from nltk.tokenize import casual_tokenize
import nltk
nltk.download('averaged_perceptron_tagger')

import pandas as pd
import psycopg2

In [3]:
import sys
sys.path.append('../')
from auth import consumer_key, consumer_secret, access_token, access_token_secret

twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret)
from aws import host as ahost, port as aport, user as auser, password as apassword, database as adatabase

connection = psycopg2.connect(host = ahost, 
                              port = aport, 
                              user = auser, 
                              password = apassword, 
                              dbname = adatabase)
cursor = connection.cursor()

In [4]:
#FOLLOWER_SAMPLE_LIMIT = 100 # number of followers to randomly sample
WORD_FREQ_LIMIT = 10 # return this number of topics that are most freq

REGULAR_NOUN_WORD_TYPE = 0
PROPER_NOUN_WORD_TYPE = 1

In [5]:
def read_try(sql):
    try:
        df = pd.read_sql(sql, con=connection)
        return pd.DataFrame() if df.empty else df
    except Exception as e:
        #print("READ ERROR", e)
        return pd.DataFrame()

def write_try(sql):
    try:
        cursor.execute(sql)  # run a psql command
        return True
    except Exception as e:
        #print("WRITE ERROR: ", e)
        return False
    finally:
        connection.commit()
        
# get all the users from the users table
def read_all_users_from_db():
    sql = 'SELECT * FROM users'
    return read_try(sql)

# get all the follower, user pairs from the followers table
def read_all_followers_from_db():
    sql = 'SELECT * FROM followers'
    return read_try(sql)

# get all the tweets from tweets table
def read_all_tweets_from_db():
    sql = 'SELECT * FROM tweets'
    return read_try(sql)

# get all word counts of tweet text
def read_all_words_from_db():
    sql = 'SELECT * FROM words'
    return read_try(sql)

# get all hashtag counts
def read_all_hashtags_from_db():
    sql = 'SELECT * FROM hashtags'
    return read_try(sql)

def read_user_from_db(user):
    sql = 'SELECT * FROM users WHERE user_handle = \'{}\''.format(user)
    return read_try(sql)

# get list of followers for a user
def read_user_followers_from_db(user, limit = 'NULL'):
    sql = 'SELECT follower_handle FROM followers WHERE user_handle = \'{}\'LIMIT {}'.format(user, limit)
    return read_try(sql)

# see if tweet is already indexed in DB
def read_tweet_from_db(tweet_id):
    sql = ('SELECT tweet_date FROM tweets '
    'WHERE tweet_id = {}'.format(tweet_id)
    )
    return read_try(sql)

# get list of word counts for a user
def read_user_words_from_db(user, limit = 'NULL', date = ''):
    sql = ('SELECT word, SUM(count) as sum_count FROM followers '
    'JOIN tweets ON tweets.follower_handle = followers.follower_handle '
    'JOIN words ON words.tweet_id = tweets.tweet_id '
    'WHERE followers.user_handle = \'{}\' GROUP BY word ORDER BY sum_count desc LIMIT {}'.format(user, limit)
    )
    return read_try(sql)

# get list of word counts for a user
def read_user_proper_words_from_db(user, limit = 'NULL', date = ''):
    sql = ('SELECT word, SUM(count) as sum_count FROM followers '
    'JOIN tweets ON tweets.follower_handle = followers.follower_handle '
    'JOIN words ON words.tweet_id = tweets.tweet_id '
    'WHERE followers.user_handle = \'{}\' '.format(user))
    sql += 'GROUP BY word HAVING MAX(words.word_type) = {} '.format(PROPER_NOUN_WORD_TYPE)
    sql += 'ORDER BY sum_count desc LIMIT {} '.format(limit)
    return read_try(sql)

# get list of hashtag counts for a user
def read_user_hashtags_from_db(user, limit = 'NULL', date = ''):
    sql = ('SELECT word, SUM(count) as sum_count FROM followers '
    'JOIN tweets ON tweets.follower_handle = followers.follower_handle '
    'JOIN hashtags ON hashtags.tweet_id = tweets.tweet_id '
    'WHERE followers.user_handle = \'{}\' GROUP BY word ORDER BY sum_count desc LIMIT {}'.format(user, limit)
    )
    return read_try(sql)

def write_user_to_db(user):
    sql = 'INSERT INTO users VALUES (\'{}\');'.format(user)
    write_try(sql)

def write_user_followers_to_db(followers, user):
    for follower in followers:
        sql = 'INSERT INTO followers VALUES (\'{}\', \'{}\');'.format(follower, user)
        write_try(sql)

In [6]:
user = "AndrewYang"
#['realDonaldTrump','ewarren','JoeBiden','SenSanders','KamalHarris','CoryBooker','AndrewYang']

In [7]:
followers = twitter.get_followers_list(screen_name = user)['users']
print(followers)

[{'id': 1169269360797634568, 'id_str': '1169269360797634568', 'name': 'Ray Morgan™', 'screen_name': 'Raymorgan254', 'location': 'Athi River, Kenya', 'description': 'love life', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 21, 'friends_count': 252, 'listed_count': 0, 'created_at': 'Wed Sep 04 15:22:20 +0000 2019', 'favourites_count': 15, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 210, 'lang': None, 'status': {'created_at': 'Sat Nov 23 11:43:55 +0000 2019', 'id': 1198205534144471041, 'id_str': '1198205534144471041', 'text': 'https://t.co/zlSrHp8mxf https://t.co/R7blgHseQW', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/zlSrHp8mxf', 'expanded_url': 'https://twitter.com/CarolRadull/status/1198195531480227840?s=19', 'display_url': 'twitter.com/CarolRadull/st…', 'indices': [0, 23]}], 'media': [{'id': 1198203918796365824

In [8]:
followers_list = []
for follower in followers:
    followers_list.append(follower['screen_name'])
print(followers_list)

['Raymorgan254', 'Iam__Rh', 'Big_Tybes', 'LucasFitzsimmo3', 'khanhkhanhhh', 'haddonfield12', 'maxswillingham', 'Gibthee', 'Mizoryy', 'mrbillceramics', 'shinvalor', 'CryptoND1', 'MandyHumbert', 'Ukudele', 'EndTime61519149', 'Tony66020649', 'Moderat24034316', 'javy79541010', 'RyanLouderMusic', '0th_Maria']


In [9]:
# add user to users db
write_user_to_db(user)
read_all_users_from_db()

,user_handle
0,test_user
1,test_user1
2,realDonaldTrump
3,AndrewYang
4,ewarren
5,JoeBiden
6,SenSanders
7,KamalHarris
8,CoryBooker


In [10]:
# add followers followers db
write_user_followers_to_db(followers_list, user)
read_all_followers_from_db()

,follower_handle,user_handle
0,test_follower1,test_user
1,test_follower2,test_user
2,test_follower1,test_user1
3,james_graziano,AndrewYang
4,rbrsq,AndrewYang
...,...,...
182,PsychoCerax,AndrewYang
183,Ch_Wieting,AndrewYang
184,Raymorgan254,AndrewYang
185,Iam__Rh,AndrewYang


In [11]:
def get_tweets_from(follower_name, results = []):
    try: 
        result = twitter.get_user_timeline(screen_name = follower_name)
        results.extend(result)
        return results
    except:
        return results

In [12]:
def get_tweet_info_from_tweets(results):
    tweets = []
    for tweet in results:
        tweet_map = {'text': tweet['text']}
        tweet_map['id'] = tweet['id']
        tweet_map['date'] = tweet['created_at']
        tweet_map['user'] = tweet['user']['screen_name']
        tweet_hashtags = []
        tweet_urls = []
        tweet_user_mentions = []
        
        entities = tweet['entities']
    
        for hashtags in entities['hashtags']:
            tweet_hashtags.append(hashtags['text'])
        for urls in entities['urls']:
            tweet_urls.append(urls['url'])
        try:
            for media in entities['media']:
                tweet_urls.append(media['url'])
        except:
            pass
        for users in entities['user_mentions']:
            tweet_user_mentions.append(users['screen_name'])
            
        tweet_map["hashtags"] = tweet_hashtags
        tweet_map["urls"] = tweet_urls
        tweet_map["user_mentions"] = tweet_user_mentions
        tweets.append(tweet_map)
        
    return tweets

In [13]:
results = []
for f in followers_list:
    results = get_tweets_from(f, results)
print(results)

[{'created_at': 'Sat Nov 23 11:43:55 +0000 2019', 'id': 1198205534144471041, 'id_str': '1198205534144471041', 'text': 'https://t.co/zlSrHp8mxf https://t.co/R7blgHseQW', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/zlSrHp8mxf', 'expanded_url': 'https://twitter.com/CarolRadull/status/1198195531480227840?s=19', 'display_url': 'twitter.com/CarolRadull/st…', 'indices': [0, 23]}], 'media': [{'id': 1198203918796365824, 'id_str': '1198203918796365824', 'indices': [24, 47], 'media_url': 'http://pbs.twimg.com/media/EKDgHaRWoAAK1mt.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EKDgHaRWoAAK1mt.jpg', 'url': 'https://t.co/R7blgHseQW', 'display_url': 'pic.twitter.com/R7blgHseQW', 'expanded_url': 'https://twitter.com/Raymorgan254/status/1198205534144471041/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 1024, 'h': 1024, 'resize': 'fit'}, 'medium': {'w': 1024, 'h': 1024, '

In [14]:
tweets_data = get_tweet_info_from_tweets(results)
tweets_data

[{'text': 'https://t.co/zlSrHp8mxf https://t.co/R7blgHseQW',
  'id': 1198205534144471041,
  'date': 'Sat Nov 23 11:43:55 +0000 2019',
  'user': 'Raymorgan254',
  'hashtags': [],
  'urls': ['https://t.co/zlSrHp8mxf', 'https://t.co/R7blgHseQW'],
  'user_mentions': []},
 {'text': "RT @TheSunFootball: Meet River Plate's sexiest fan @MelisiaOficial - who said she would auction off her underwear after a historic win http…",
  'id': 1198183874485993472,
  'date': 'Sat Nov 23 10:17:51 +0000 2019',
  'user': 'Raymorgan254',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['TheSunFootball', 'MelisiaOficial']},
 {'text': "RT @jahkid_dan: If you can't dance when drinking alcohol, at least speak English or promise people jobs do something when drunk don't waste…",
  'id': 1197353202687066113,
  'date': 'Thu Nov 21 03:17:03 +0000 2019',
  'user': 'Raymorgan254',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['jahkid_dan']},
 {'text': 'RT @CFCBrian_: The Special One. https://t.co/ghty7NbzWU',

POS tag each word using NLTK   
Resources:
- [Categorizing and Tagging Words](https://www.nltk.org/book/ch05.html)
- [NLTK tags](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)

Noun tags:
- NN noun, singular 'desk'
- NNS noun plural 'desks'
- NNP proper noun, singular 'Harrison'
- NNPS proper noun, plural 'Americans'

In [15]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        return True if s.isalpha() else False
    except UnicodeDecodeError:
        return False

In [16]:
# noun based word extraction
def word_hashtag_extraction_noun_based(tweet_dictionary):  
    text = tweet_dictionary['text']
    # skip over retweets
    if text[0:4] == 'RT @':
        return [], []
    text = casual_tokenize(tweet_dictionary['text'])
    result = nltk.pos_tag(text)
    
    hashtags = ['#' + s for s in tweet_dictionary['hashtags']]
    urls = tweet_dictionary['urls']
    user_mentions = ['@' + s for s in tweet_dictionary['user_mentions']]
    ignore = hashtags + urls + user_mentions
    
    all_noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    proper_noun_tags = ['NNP', 'NNPS']
    
    cleaned_text = {}
    for t in result:
        word = t[0]
        tag = t[1]
        if (tag in all_noun_tags) and (not word in ignore) and isEnglish(word):
            if tag in proper_noun_tags:
                cleaned_text[word.lower()] = PROPER_NOUN_WORD_TYPE 
            else:
                cleaned_text[word.lower()] = REGULAR_NOUN_WORD_TYPE 
            
    return cleaned_text, set([h.lower() for h in hashtags])

In [17]:
test_tweet = tweets_data[2]
print(test_tweet)
words, tags = word_hashtag_extraction_noun_based(test_tweet)
print(words)
print(tags)

{'text': "RT @jahkid_dan: If you can't dance when drinking alcohol, at least speak English or promise people jobs do something when drunk don't waste…", 'id': 1197353202687066113, 'date': 'Thu Nov 21 03:17:03 +0000 2019', 'user': 'Raymorgan254', 'hashtags': [], 'urls': [], 'user_mentions': ['jahkid_dan']}
[]
[]


In [18]:
def get_freq_map(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [19]:
word_count = get_freq_map(words)
print(word_count)
hashtag_count = get_freq_map(tags)
print(hashtag_count)

{}
{}


In [20]:
for w,c in word_count.items():
    print(w,c)

In [21]:
def write_tweets_words_hashtags_to_db(tweets_dictionary):
    for tweet in tweets_dictionary:
        follower = tweet['user']
        tweet_id = tweet['id']
        tweet_date = tweet['date']
        sql = 'INSERT INTO tweets VALUES ({},\'{}\',\'{}\');'.format(tweet_id, follower, tweet_date)

        # if writing tweet to database was sucessful, then we need to get write word count info
        if write_try(sql): # or True:
            words, tags = word_hashtag_extraction_noun_based(tweet)
            word_count = get_freq_map(words)
            hashtag_count = get_freq_map(tags)
            
            for w, c in word_count.items():
                word_type_int = words[w]
                sql2 = 'INSERT INTO words VALUES ({},\'{}\',{},{});'.format(tweet_id, w, c, word_type_int)
                write_try(sql2)

            for w, c in hashtag_count.items():
                sql2 = 'INSERT INTO hashtags VALUES ({},\'{}\',{});'.format(tweet_id, w, c)
                write_try(sql2)

In [22]:
write_tweets_words_hashtags_to_db(tweets_data)

In [23]:
all_tweets = read_all_tweets_from_db()
print("All tweets in tweets DB")
print(all_tweets)

all_words = read_all_words_from_db()
print("All words in words DB")
print(all_words)

all_hashtags = read_all_hashtags_from_db()
print("All hashtags in hashtags DB")
print(all_hashtags)

All tweets in tweets DB
                 tweet_id follower_handle          tweet_date
0                       0  test_follower1 1977-01-08 04:05:06
1                       1  test_follower1 1999-01-08 04:05:06
2     1198375711133884416    banks_mikado 2019-11-23 23:00:08
3     1198375272934002688    banks_mikado 2019-11-23 22:58:24
4     1198372948207448065    banks_mikado 2019-11-23 22:49:10
...                   ...             ...                 ...
1185  1195116626724700160       Big_Tybes 2019-11-14 23:09:42
1186  1195116134732713985       Big_Tybes 2019-11-14 23:07:45
1187  1194856023388897280       Big_Tybes 2019-11-14 05:54:09
1188  1194142517718962176       Big_Tybes 2019-11-12 06:38:56
1189  1194141208185991168       Big_Tybes 2019-11-12 06:33:44

[1190 rows x 3 columns]
All words in words DB
                 tweet_id       word  count  word_type
0     1198375711133884416   birthday      1          0
1     1198372948207448065   birthday      1          0
2     11978413114344

In [24]:
user_words_df = read_user_words_from_db(user, limit = WORD_FREQ_LIMIT)
user_words_better_df = read_user_proper_words_from_db(user, limit = WORD_FREQ_LIMIT)
user_hashtags_df = read_user_hashtags_from_db(user, limit = WORD_FREQ_LIMIT)

print("List of words that followers of {} talk about".format(user))
print(user_words_df)

print("List of proper noun(?) words that followers of {} talk about".format(user))
print(user_words_better_df)

print("List of hashtags that followers of {} use".format(user))
print(user_hashtags_df)

List of words that followers of AndrewYang talk about
       word  sum_count
0  november          6
1         s          5
2       art          4
3     world          4
4         t          4
5       lol          3
6  prophecy          3
7  children          3
8   strange          3
9       sec          3
List of proper noun(?) words that followers of AndrewYang talk about
       word  sum_count
0  november          6
1         s          5
2     world          4
3       art          4
4        so          3
5  prophecy          3
6       lol          3
7  children          3
8   strange          3
9       man          3
List of hashtags that followers of AndrewYang use
             word  sum_count
0         #mlconf          3
1        #blocked          1
2           #hack          1
3           #arts          1
4            #xrp          1
5       #powerful          1
6  #gretathunberg          1
7        #stellar          1
8        #artwork          1
9     #cryptoland          1


In [25]:
word_results = word_count_results = tag_results = tag_count_results = []
if user_words_df.size > 0:
    word_results = user_words_df["word"].tolist()
    word_count_results = user_words_df["sum_count"].tolist()
if user_hashtags_df.size > 0:
    tag_results = user_hashtags_df["word"].tolist()
    tag_count_results = user_hashtags_df["sum_count"].tolist()
    
final_results = {'token_labels': word_results,
        'token_counts': word_count_results,
        'hash_labels': tag_results,
        'hash_counts': tag_count_results}
print(final_results)

{'token_labels': ['november', 's', 'art', 'world', 't', 'lol', 'prophecy', 'children', 'strange', 'sec'], 'token_counts': [6, 5, 4, 4, 4, 3, 3, 3, 3, 3], 'hash_labels': ['#mlconf', '#blocked', '#hack', '#arts', '#xrp', '#powerful', '#gretathunberg', '#stellar', '#artwork', '#cryptoland'], 'hash_counts': [3, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
